# Génération de texte avec réseaux de neurones récurrents LSTM avec Keras

#### Par : BOUTAHIR Mohamed Khalifa [ MASTER IPS ]
##### Data : nous allons utiliser un livre de l'enfance comme Data: "Alice's Adventures in Wonderland de Lewis Carroll" . https://gist.github.com/phillipj/4944029
##### Le but de projet est de predire une suite où un nouveau scenario pour ce livre .L'utilisation des réseaux LSTM est pour apprendre des séquences de personnages d’Alice au pays des merveilles. Et par la suite le modèle va générer des nouvelles séquences de caractères.
<img src="wonderland.jpg" width=400 />

## 1- Importation des bibliothèques et le jeu de données

In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


# 2- Préparation de l’ensemble de données

#### Importation du fichier texte et le convertir en minuscule

In [2]:
filename = "wonderland.txt"
text = open(filename).read()
text = text.lower()

#### Detecter les caractères de jeu de donnees et les classer 
 Nous ne pouvons pas modéliser les caractères directement dans le réseau de neurones, nous devons plutôt convertir les caractères en entiers.

In [11]:
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [12]:
print(char_to_int)

{'m': 32, 'd': 23, 'l': 31, 't': 39, '\n': 0, 'r': 37, 'p': 35, '?': 15, '0': 11, 'o': 34, 'x': 43, 'v': 41, '!': 2, 'a': 20, 'k': 30, '(': 5, '"': 3, 'e': 24, 'n': 33, 'h': 27, '`': 19, 'g': 26, 'z': 45, 'i': 28, "'": 4, 'q': 36, ']': 17, ' ': 1, ',': 8, '.': 10, 'j': 29, 'c': 22, 'u': 40, '*': 7, '[': 16, ';': 14, '3': 12, ')': 6, 's': 38, '_': 18, 'f': 25, ':': 13, 'w': 42, 'y': 44, 'b': 21, '-': 9}


#### Calcule le nombre total des caractères  de texte et le nombre des caractères 

In [13]:
n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  148574
Total Vocab:  46


# 2- Creation de réseau de neurones

Nous allons diviser le texte du livre en sous-séquences d'une longueur fixe de 100 caractères. <br>
Chaque modèle d'apprentissage du réseau est composé de 100 pas de temps d'un caractère (X) suivis d'une sortie de caractère (y).

In [23]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

On va calculer le nombre des échantillons qu'on dans le "X"

In [24]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  148474


Maintenant que nous avons préparé nos données d’entraînement, nous devons les transformer pour qu’elles puissent être utilisées avec Keras.

Nous devons d’abord transformer la liste des séquences d’entrée sous la forme [échantillons, pas de temps, fonctions] pour qu'on puisse les entrés pour notre réseau LSTM.

In [31]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

X = X / float(n_vocab)

In [30]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

<table>
    <tr>
        <td>Platform d'essai</td>
        <td>Temps en (s) pour chaque itération</td>
    </tr>
    <tr>
        <td>GOOGLE COLAB</td>
        <td>850s</td>
    </tr>
    <tr>
        <td>MY LAPTOP</td>
        <td>2000s</td>
    </tr>
    <tr>
        <td>COCALC : https://cocalc.com</td>
        <td>5000s</td>
    </tr>
    <tr>
        <td>AZURE NOTEBOOK</td>
        <td>24394s</td>
    </tr>
</table>

GOOGLE COLAB itération : 
    <img src="capture3.png">

AZURE itération :
    <img src="capture.png">

COLALC itération : 
    <img src="capture2.png">

In [39]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" 

  just then her head struck against the roof of the hall:  in
fact she was now more than nine feet "
 hare    `and the ture oi the toies oi the sooel ' 
  `thet did toe tore the sime!' she manch hare taid to alice, and the tart on thine the was oo the was wottenn to the thet was ano ariel and then so her hend and the tabde so sal at the could, and she taid to she care pateng toiee an the woode and all the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the 
Done.
